In [1]:
import numpy as np
import os
import pypdb
import pandas as pd
import urllib.request
import xmltodict

In [36]:
def load_dataframe(file) -> pd.DataFrame:
    df = pd.read_csv(file, keep_default_na=False, na_values=pd.NA)
    df.rename(columns={'1st protein hit': 'Top_PDB'}, inplace=True)
    df['Top_PDB'] = df['Top_PDB'].apply(lambda x: x.split('.')[0])
    df['PDB_Annotation'] = pd.NA
    df['Keywords'] = pd.NA
    
    return df

def make_url (uniprot_url_base, accession) -> str:
    url = uniprot_url_base + f'{accession}'
    return url

def format_afdb_df(dt) -> pd.DataFrame:
    '''Renames columns referenceing PDB Database. These references in the AFDB files are due to
    the same code being used to recover the top hits from both PDB and AFDB databases.
    
    Also cleans the 'Top_AFDB' text so the accession number is all that remains.
    '''
    dt.rename(columns={'Top_PDB': 'Top_AFDB', 'Keywords': 'URL', 
                       'PDB_Annotation': 'AFDB_Annotation'}, inplace=True)
    dt['Hits'] = dt['Hits'].replace('No hits on PDB', 'No Hits in AFDB')
    
    # Cleaning accessions where they exist, as some are empty.
    for i in dt[(dt['Top_AFDB'] != '')].index:
        dt.loc[i, 'Top_AFDB'] = dt.loc[i, 'Top_AFDB'].split('-')[1]
    
    return dt

def get_uniprot_annotation(url) -> str:
    
    with urllib.request.urlopen(url) as u:
        data = u.read().decode('utf-8')
    
    #  Handle the case where there is no XML file
    if len(data) == 0:
        annotation = 'ERROR: MISSING XML FILE!!!'
        return annotation
    
    #  If there is an XML file, it gets parsed into something readable
    data = xmltodict.parse(data)
    
    #  Handle error where an outdated entry has an XML file without any actual entry
    if 'entry' not in data['uniprot'].keys():
        annotation = 'XML Error! Possible outdated entry.'
        return annotation
    
    #  Handle error where by this is no 'protein' entry.
    if 'protein' not in data['uniprot']['entry'].keys():
        annotation = 'XML Error! No information Found.'
        return annotation    
    
    #  Now, with a seemingly correct XML, we pull the actual annotation
    if 'submittedName' in data['uniprot']['entry']['protein'].keys():
        if type(data['uniprot']['entry']['protein']['submittedName']) == list:
            annotation = data['uniprot']['entry']['protein']['submittedName'][0]['fullName']['#text']
        else:
            if type(data['uniprot']['entry']['protein']['submittedName']['fullName']) != str:
                annotation = data['uniprot']['entry']['protein']['submittedName']['fullName']['#text']
            if type(data['uniprot']['entry']['protein']['submittedName']['fullName']) == str:
                annotation = data['uniprot']['entry']['protein']['submittedName']['fullName']
    
    if 'recommendedName' in data['uniprot']['entry']['protein'].keys():
        if type(data['uniprot']['entry']['protein']['recommendedName']['fullName']) != str:
            annotation = data['uniprot']['entry']['protein']['recommendedName']['fullName']['#text']
        if type(data['uniprot']['entry']['protein']['recommendedName']['fullName']) == str:
            annotation = data['uniprot']['entry']['protein']['recommendedName']['fullName']
    
    if not annotation:
        annotation = 'None Found'
        #print('no annotation found for entry.')
    
    return annotation

def add_pdb_annotation(df) -> pd.DataFrame:
    annotated_list = df[~(df['Top_PDB'] == '')].index
    print('Found %i entries to query for PDB annotation.' % len(annotated_list))
    da = df.copy()
    count = 0
          
    for i in annotated_list:
        cif_ref = da.loc[i, 'Top_PDB']
        pdb_json = pypdb.get_all_info(cif_ref)
        pdb_title = pdb_json['struct_keywords']['pdbx_keywords']
        pdb_keywords = pdb_json['struct_keywords']['text']
        da.loc[i, 'PDB_Annotation'] = pdb_title
        da.loc[i, 'Keywords'] = pdb_keywords
        count += 1
        if count % 10 == 0:
          print(count, end='')
        else:
          print('.', end='')
        
    return da

def add_uniprot_annotation(dt) -> pd.DataFrame:
    # Frist we get the entries that have hits.
    to_annotate = dt[(dt['Top_AFDB'] != '')].shape[0]
    print('\nFound %i entries to query for AFDB (Uniprot) annotation.' % to_annotate)
    count = 0
    for i in dt[(dt['Top_AFDB'] != '')].index:
        url = make_url(uniprot_url_base, dt.loc[i, 'Top_AFDB'])
        dt.loc[i, 'URL'] = url
        annotation = get_uniprot_annotation(url + '.xml')
        dt.loc[i, 'AFDB_Annotation'] = annotation
        count += 1
        if count % 10 == 0:
          print(count, end='')
        else:
          print('.', end='')
    return dt

def annotate_all(foldseek_PDB_results, foldseek_AFDB_results) -> (pd.DataFrame, pd.DataFrame):
    #  Here we load the results from the different databases. The 'add_XXX_annotation' lines include
    #  calls to websites to retrieve annotation information
    dp = load_dataframe(foldseek_PDB_results) # The PDB results and DataFrame formating
    dp = add_pdb_annotation(dp)
    dp = dp.mask(dp == '').replace(pd.NA, np.nan)
    dp.set_index('Sequence', inplace=True)

    da = load_dataframe(foldseek_AFDB_results) # The AFDB results and DataFrame formating
    da = format_afdb_df(da)
    da = add_uniprot_annotation(da)
    da = da.mask(da == '').replace(pd.NA, np.nan)
    da.set_index('Sequence', inplace=True)
    
    return dp, da

def report_results(dp, da):
    #Setup the final DataFrame (df) and parse the results from above:
    df = pd.DataFrame(index=dp.sort_index().index, columns=['Annotation', 'Organism', 'Database'])
    for i in dp.index:
        if dp.loc[i, :].isna()[2]:  # If there is no PDB annotation
            if da.loc[i, :].isna()[2]: # ...and there is no AFDB annotation
                df.loc[i, 'Annotation'] = 'unknown'
            else:
                df.loc[i, 'Annotation'] = da.loc[i, 'AFDB_Annotation']
                df.loc[i, 'Database'] = 'AFDB/Uniprot'
                df.loc[i, 'Organism'] = da.loc[i, '1st Specie']
        else:  # If a PDB annotation exists, it is presumed to be the most reliable
            df.loc[i, 'Annotation'] = dp.loc[i, 'PDB_Annotation']
            df.loc[i, 'Database'] = 'RSCB/PDB'
            df.loc[i, 'Organism'] = dp.loc[i, '1st Specie']
     
    print('\n\nSummary of identified functions')
    print(df.iloc[:, 0].value_counts().head(50))
    
    with pd.ExcelWriter(output_file) as writer:
        #  The combined annotation first
        df.to_excel(writer, sheet_name="Combined_Annotations")
        #  Seperate sheet for individual PDB and AFDB results
        dp.to_excel(writer, sheet_name="PDB_results")
        da.to_excel(writer, sheet_name="AFDB_results")

### Set the parameters:
#### PDB & AFDB input files. Desired output file name.

In [33]:
foldseek_PDB_results = '../FS_summary_pdb.csv'  # FoldSeek PDB Results File
foldseek_AFDB_results = '../FS_summary_afdb.csv'  # Foldseek AFDB Results File
output_file = '../Combined_Anotations_MVB.xlsx'
uniprot_url_base = 'https://www.uniprot.org/uniprot/'

# Main Program:
### annotate_all: Searches PDB and Uniprot databases, seperately, for annotations for each structure
### report_results: Combines results, prints a short summary and write a detailed Excel file.

In [34]:
pdb_annotations, afdb_annotation = annotate_all(foldseek_PDB_results, foldseek_AFDB_results)
report_results(pdb_annotations, afdb_annotation)

Found 362 entries to query for PDB annotation.
.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360..
Found 515 entries to query for AFDB (Uniprot) annotation.
https://www.uniprot.org/uniprot/K0F4M0
.https://www.uniprot.org/uniprot/Q8ZP99
.https://www.uniprot.org/uniprot/A0A0K0DWK1
.https://www.uniprot.org/uniprot/A0A077ZDV8
.https://www.uniprot.org/uniprot/Q4E3L0
.https://www.uniprot.org/uniprot/B6SF70
.https://www.uniprot.org/uniprot/Q8ZP99
.https://www.uniprot.org/uniprot/Q8CYI1
.https://www.uniprot.org/uniprot/A0A5P3G0W5
.https://www.uniprot.org/uniprot/A4ICS0
10https://www.uniprot.org/uniprot/Q9I1K4
.https://www.unip